In [0]:
aws_keys_df = spark.read.format('csv').option('header','true').option('inferschema','true').load('/FileStore/tables/s3bucket_accessKeys.csv')

In [0]:
aws_keys_df.columns

Out[2]: ['Access key ID', 'Secret access key']

In [0]:
ACCESS_KEY = aws_keys_df.select('Access key ID').take(1)[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').take(1)[0]['Secret access key']

In [0]:
import urllib

ENCODED_SECRET_KEY = urllib.parse.quote(string = SECRET_KEY,safe ="")

In [0]:
AWS_S3_BUCKET = 'projectsdata'
MOUNT_NAME = '/mnt/mount_s3'

SOURCE_URL = "s3a://%s:%s@%s" %(ACCESS_KEY,ENCODED_SECRET_KEY,AWS_S3_BUCKET)

In [0]:
dbutils.fs.mount(SOURCE_URL,MOUNT_NAME)

Out[6]: True

In [0]:
%fs ls '/mnt/mount_s3'

path,name,size,modificationTime
dbfs:/mnt/mount_s3/employee_attrition/,employee_attrition/,0,0


In [0]:
aws_employeeAttrition_df = spark.read.format('csv').option('header','true').option('inferschema','true').load('/mnt/mount_s3/employee_attrition/')

In [0]:
aws_employeeAttrition_df.createOrReplaceTempView("temp")

In [0]:
%sql select sum(EmployeeCount) from temp

sum(EmployeeCount)
1470


In [0]:
%sql select 
case when age between 20 and 25 then '20-25' when age between 26 and 32 then '26-32' when age between 33 and 40 then '33-40' else '40+'
end age_group, sum(EmployeeCount) as total_leaving from temp where Attrition = 'Yes' group by 1 order by 1

age_group,total_leaving
20-25,34
26-32,85
33-40,56
40+,62


Databricks visualization. Run in Databricks to view.

In [0]:
# This definitely shows that least no of employees leaving the organization belongs to the age group 20-25. Thus, organisation should target other age group people issues first and issues related to people at juniormost level can be tackled later  

In [0]:
%sql select Department,sum(EmployeeCount) as total_leaving from temp where Attrition = 'Yes' group by Department

Department,total_leaving
Sales,92
Research & Development,133
Human Resources,12


Databricks visualization. Run in Databricks to view.

In [0]:
# most of the employees leaving the organization belongs to the Research & Development and Sales department. Organisation should do proper check of these departments

In [0]:
%sql select Gender,sum(EmployeeCount) as total_leaving from temp where Attrition = 'Yes' group by Gender

Gender,total_leaving
Female,87
Male,150


Databricks visualization. Run in Databricks to view.

In [0]:
# no of males leaving the organisation are almost double than the no of females leaving the organisation. There can be various possibilities - 
                #either the no of females working in the company are relatively less than males and hence, overall results are also inclined towards males (ratioWise)                     OR
                #this organisation provides some really good gender specific benefits to its female employees, so females wish to retain here                                              OR
                #Industry is more biased towards males and Females are not getting equal opportunities outside. So they have to stay in the current organisation                            OR 
                #Females are having less decision making power and thus they can not leave the organisation even when the organisation is following some corrupt practices

In [0]:
%sql select JobRole,avg(JobSatisfaction) as JobRole_Satisfaction_level from temp group by JobRole

JobRole,JobRole_Satisfaction_level
Sales Executive,2.754601226993865
Manufacturing Director,2.682758620689655
Laboratory Technician,2.6911196911196913
Sales Representative,2.7349397590361444
Healthcare Representative,2.786259541984733
Research Scientist,2.7739726027397262
Manager,2.7058823529411766
Research Director,2.7
Human Resources,2.5576923076923075


In [0]:
%sql select JobRole,sum(EmployeeCount) as total_leaving from temp where Attrition = 'Yes' group by JobRole

JobRole,total_leaving
Sales Executive,57
Manufacturing Director,10
Laboratory Technician,62
Sales Representative,33
Healthcare Representative,9
Research Scientist,47
Manager,5
Research Director,2
Human Resources,12


Databricks visualization. Run in Databricks to view.

In [0]:
# Job for Human Resources is of least satisfaction level i.e. 2.5, all other job roles are having almost same satisfaction level. But majorly people, who are leaving, are either Lab Technician or Research Scientist or Sales Executive. This definitely shows that dissatisfaction with the job is not the reason for the employees to leave the organisation

In [0]:
%sql select 
case when MonthlyIncome between 0 and 3999 then '0-3999' when MonthlyIncome between 4000 and 7999 then '4000-7999' when MonthlyIncome between 8000 and 14999 then '8000-14999' else '15000+'
end MonthlyIncome_groups, sum(EmployeeCount) as total_leaving from temp where Attrition = 'Yes' group by 1 order by 1

MonthlyIncome_groups,total_leaving
0-3999,137
15000+,5
4000-7999,60
8000-14999,35


Databricks visualization. Run in Databricks to view.

In [0]:
# People earning very high salary will be very less and people earning low salary will be ofcourse large in number but still this data analysis show that attrition rate of people coming under the lowest bracket salaries is significantly higher as compared to the number of people working in that bracket. Thus, low monthly income can be considered as one of the reason for the employees to leave the organisation  